In [47]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler

datapath  = 'Data/DataSet.xlsx'

df = pd.read_excel(datapath, sheet_name='FF')

In [48]:
df['CITY'] = df['CITY'].str.lower()
df['CITY'] = df['CITY'].str.strip()
df['CITY'] = df['CITY'].replace('kathmandhu', 'kathmandu')
df['CITY'] = df['CITY'].replace('sitapaila', 'kathmandu')


In [49]:
# Calculate number of rows before dropping
num_rows_before = df.shape[0]

# Remove null values
df = df.dropna(subset=['PRICE_N'])

# Calculate IQR
Q1 = df['PRICE_N'].quantile(0.25)
Q3 = df['PRICE_N'].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as values below Q1-1.5*IQR or above Q3+1.5*IQR
outliers = df[(df['PRICE_N'] < (Q1 - 1.5 * IQR)) | (df['PRICE_N'] > (Q3 + 1.5 * IQR))].index

# Drop the outliers
df = df.drop(outliers)

# Calculate number of rows after dropping
num_rows_after = df.shape[0]

# Calculate and print number of rows dropped
num_rows_dropped = num_rows_before - num_rows_after
print(f"Number of rows dropped for price outlier: {num_rows_dropped}")

Number of rows dropped for price outlier: 156


In [50]:
# Calculate number of rows before dropping
num_rows_before = df.shape[0]

# Drop rows where 'CITY' is 'undefined'
df = df[df['CITY'] != 'undefined']

# Calculate number of rows after dropping
num_rows_after = df.shape[0]

# Calculate and print number of rows dropped
num_rows_dropped = num_rows_before - num_rows_after
print(f"Number of rows dropped for undefined cities: {num_rows_dropped}")

unique_cities = df['CITY'].unique()
print(unique_cities)

Number of rows dropped for undefined cities: 2
['lalitpur' 'kathmandu' 'bhaktapur' 'rupandehi' 'kaski' 'dhading'
 'chitwan' 'nuwakot' 'banke' 'sunsari' 'kavrepalanchok' 'jhapa' 'dang']


In [51]:
num_sunsari = df['CITY'].value_counts()['sunsari']
print(f"Number of rows with 'sunsari': {num_sunsari}")
# Identify the rows to drop
rows_to_drop = df[~df['CITY'].isin(['kathmandu', 'lalitpur', 'bhaktapur'])].index

# Drop the rows
df = df.drop(rows_to_drop)

Number of rows with 'sunsari': 1


In [52]:
# Count the number of rows with each unique city
city_counts = df['CITY'].value_counts()

# Print the city counts
print(city_counts)

CITY
kathmandu    1304
lalitpur      695
bhaktapur      42
Name: count, dtype: int64


In [53]:
# Create a new column with the mean price for each city
mean_prices = df.groupby('CITY')['PRICE_N'].mean()

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the mean prices and transform the mean prices
normalized_prices = scaler.fit_transform(mean_prices.values.reshape(-1, 1))

# Create a mapping from city to normalized mean price
mapping = dict(zip(mean_prices.index, normalized_prices.flatten()))

# Apply the mapping to the 'CITY' column
df['CITY'] = df['CITY'].map(mapping)

# Print the mapping
print(mapping)

# Save the mapping to a pickle file
with open('city_mapping.pkl', 'wb') as f:
    pickle.dump(mapping, f)

{'bhaktapur': 0.0, 'kathmandu': 1.0, 'lalitpur': 0.22309048700191525}
